# Use metadata from multiple templates for fdnc

In [15]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from DLC_for_WBFM.utils.xinwei_fdnc.formatting import zimmer2leifer
from DLC_for_WBFM.utils.xinwei_fdnc.predict import load_fdnc_options_and_template, track_using_fdnc
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from DLC_for_WBFM.utils.projects.finished_project_data import ProjectData
fname = "/scratch/zimmer/Charles/dlc_stacks/worm3-2021_09_29/project_config.yaml"
# fname = "/scratch/zimmer/Charles/dlc_stacks/project-pytest/project_config.yaml"
project_data = ProjectData.load_final_project_data_from_config(fname)

Project data for directory:
/scratch/zimmer/Charles/dlc_stacks/worm3-2021_09_29 
Found the following raw data files:
red_data: True
green_data: True
============Segmentation===============
raw_segmentation: True
segmentation: False
============Tracklets==================
df_training_tracklets: True
reindexed_masks_training: True
============Traces=====================
red_traces: False
green_traces: False
final_tracks: True
behavior_annotations: True



In [17]:
import pandas as pd
from collections import defaultdict
from tqdm.auto import tqdm
from fDNC.src.DNC_predict import pre_matt, predict_matches, filter_matches
from DLC_for_WBFM.utils.projects.utils_neuron_names import int2name

def track_using_fdnc_ONLY_TRAINING(project_dat: ProjectData,
                                   template_pos,
                                   prediction_options,
                                   match_confidence_threshold,
                                   num_frames):
    # Loop through detections and match all to template

    # Initialize
#     num_frames = project_dat.num_frames - 1
    coords = ['z', 'x', 'y', 'likelihood']

    sz = (num_frames, len(coords))
    neuron_arrays = defaultdict(lambda: np.zeros(sz))
    all_matches = []
    for i_frame in tqdm(range(num_frames), total=num_frames, leave=False):

        pts = project_dat.get_centroids_as_numpy_training(i_frame)
        pts_scaled = zimmer2leifer(pts)
        # Match
        matches = predict_matches(test_pos=pts_scaled, template_pos=template_pos, **prediction_options)
        matches = filter_matches(matches, match_confidence_threshold)

        # For each match, save location
        for m in matches:
            this_unscaled_pt = pts[m[1]]
            this_template_idx = m[0]

            neuron_arrays[this_template_idx][i_frame, :3] = this_unscaled_pt
            neuron_arrays[this_template_idx][i_frame, 3] = m[2]  # Match confidence
        all_matches.append(matches)

    # Convert to pandas multiindexing formatting
    new_dict = {}
    for i_template, data in neuron_arrays.items():
        for i_col, coord_name in enumerate(coords):
            # Note: these neuron names are final for all subsequent steps
            k = (int2name(i_template + 1), coord_name)
            new_dict[k] = data[:, i_col]

    df = pd.DataFrame(new_dict)

    return df, all_matches

def generate_template(i):
    custom_template = project_data.get_centroids_as_numpy_training(i)
    return zimmer2leifer(custom_template)

In [18]:
num_templates = project_data.reindexed_metadata_training.num_frames
all_templates = [generate_template(i) for i in range(num_templates)]

base_prediction_options, _, _ = load_fdnc_options_and_template()

match_confidence_threshold = 0.3

In [19]:
df_per_template = []
matches_per_template = []
for template in tqdm(all_templates):
    df, matches = track_using_fdnc_ONLY_TRAINING(project_data, template, base_prediction_options, match_confidence_threshold, num_frames=num_templates)
    df_per_template.append(df)
    matches_per_template.append(matches)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
# Process into single set of matches
from scipy.optimize import linear_sum_assignment

In [22]:
# linear_sum_assignment(all_final_conf[0].toarray())

In [23]:
import numpy as np
from scipy.sparse import coo_matrix

In [24]:
# Note: the second index in the match is the template
# Combine the matches from each template by summing the confidence

# First, convert each to a sparse matrix
# i_match = 0
all_final_conf = []
all_individual_conf = []
shape = (130, 130)

for i_match in range(num_templates):
    all_conf = []

    for i_template in range(num_templates):
        m = np.array(matches_per_template[i_template][i_match])
        row, col, data = m[:,0], m[:,1], m[:,2]
    #     shape = (int(max(row))+1, int(max(col))+1)
        conf = coo_matrix((data, (row, col)), shape=shape, dtype=float)
        all_conf.append(conf)
    final_conf = sum(all_conf)
    
    all_individual_conf.append(all_conf)
    all_final_conf.append(final_conf)
#     other_frame_conf = sum(all_conf[1:])


# Visualize the matches (2d)

In [25]:
# i_test = 0

# plt.figure(figsize=(25, 15))
# plt.imshow(all_final_conf[i_test].toarray())
# plt.title("Accuracy for 2 template frames")

In [26]:
from ipywidgets import interact
def f(i_test):
    %matplotlib inline
    plt.figure(figsize=(25, 15))
    plt.imshow(all_final_conf[i_test].toarray())
    plt.title("Accuracy all templates")
    plt.colorbar()
    
    plt.show()

interact(f, i_test=(0,9))

interactive(children=(IntSlider(value=4, description='i_test', max=9), Output()), _dom_classes=('widget-intera…

<function __main__.f(i_test)>

In [62]:
from ipywidgets import interact
def f(i_template, i_test):
    %matplotlib inline
    plt.figure(figsize=(25, 15))
    plt.imshow(all_individual_conf[i_test][i_template].toarray())
    plt.title("Accuracy for 1 template frame")
    plt.colorbar()
    
    plt.show()

interact(f, i_template=(0,9), i_test=(0,9))

interactive(children=(IntSlider(value=4, description='i_template', max=9), IntSlider(value=4, description='i_t…

<function __main__.f(i_template, i_test)>

# Summary functions

In [58]:
from DLC_for_WBFM.utils.performance.comparing_ground_truth import calc_summary_scores_for_training_data
from DLC_for_WBFM.utils.xinwei_fdnc.formatting import flatten_nested_list
from DLC_for_WBFM.utils.feature_detection.utils_networkx import calc_bipartite_from_candidates

In [61]:
for i in range(10):
    m = matches_per_template[i]
    m = flatten_nested_list(m)
    m = np.array(m)
    m[:,2] /= 10
    matches, conf, _ = calc_bipartite_from_candidates(m, min_conf=0.1)
    m_final = [(m[0],m[1], c) for m, c in zip(matches, conf)]

    num_tp, num_outliers, num_missing, max_possible = calc_summary_scores_for_training_data(m_final)

    print(f"For pose {i}, tp: {num_tp / max_possible: .2f}, outliers: {num_outliers / max_possible: .2f}")

For pose 0, tp:  1.00, outliers:  0.00
For pose 1, tp:  1.00, outliers:  0.00
For pose 2, tp:  1.00, outliers:  0.00
For pose 3, tp:  1.00, outliers:  0.00
For pose 4, tp:  1.00, outliers:  0.00
For pose 5, tp:  1.00, outliers:  0.00
For pose 6, tp:  0.97, outliers:  0.01
For pose 7, tp:  0.97, outliers:  0.00
For pose 8, tp:  0.93, outliers:  0.03
For pose 9, tp:  0.85, outliers:  0.06


# Visualize the matches (in zxy)

In [32]:
from DLC_for_WBFM.utils.visualization.visualization_tracks import visualize_tracks

In [52]:
pair = (0, 9)

n0, n1 = project_data.get_centroids_as_numpy_training(pair[0]), project_data.get_centroids_as_numpy_training(pair[1])
n0, n1 = zimmer2leifer(n0), zimmer2leifer(n1)

# Treat the first of the pair as the template
m = matches_per_template[pair[0]][pair[1]]

In [53]:
visualize_tracks(n0, n1, m)

[LineSet with 77 lines.,
 PointCloud with 120 points.,
 PointCloud with 120 points.]

# Debugging actual function

In [82]:
from DLC_for_WBFM.utils.xinwei_fdnc.predict import track_using_fdnc_from_config
from DLC_for_WBFM.utils.projects.utils_filepaths import ModularProjectConfig
project_path = "/scratch/zimmer/Charles/dlc_stacks/worm3-multiple_templates/project_config.yaml"

project_cfg = ModularProjectConfig(project_path)
tracks_cfg = project_cfg.get_tracking_config()

track_using_fdnc_from_config(project_cfg, tracks_cfg, True)

Project data for directory:
/scratch/zimmer/Charles/dlc_stacks/worm3-multiple_templates 
Found the following data files:
red_data: True
green_data: True
============Segmentation===============
raw_segmentation: True
segmentation: False
============Tracklets==================
df_training_tracklets: True
reindexed_masks_training: True
============Traces=====================
red_traces: False
green_traces: False
final_tracks: True
behavior_annotations: False



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
%debug

# Multiprocessing version

In [41]:
import concurrent.futures
import numpy as np

In [42]:
num_templates = 10
all_templates = [generate_template(i) for i in range(num_templates)]

base_prediction_options = load_prediction_options()

match_confidence_threshold = 0.3

In [45]:
%%time
df_per_template = []
matches_per_template = []

def _parallel_func(template):
#     print("Starting to track...")
    base_prediction_options['template_pos'] = template
    df, matches = track_using_fdnc_ONLY_TRAINING(project_data, base_prediction_options, match_confidence_threshold, num_frames=num_templates)
    return df, matches

# with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
with concurrent.futures.ThreadPoolExecutor() as executor:
    submitted_jobs = [executor.submit(_parallel_func, template) for template in all_templates]
    results = [job.result() for job in submitted_jobs]
    
    for r in results:
        df_per_template.append(r[0])
        matches_per_template.append(r[1])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 15min 7s, sys: 17min 22s, total: 32min 29s
Wall time: 48.3 s


In [62]:
%%time
df_per_template = []
matches_per_template = []

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    submitted_jobs = [executor.submit(_parallel_func, template) for template in all_templates]
    results = [job.result() for job in submitted_jobs]
    
    for r in results:
        df_per_template.append(r[0])
        matches_per_template.append(r[1])
print(len(matches_per_template))
print(len(matches_per_template[0]))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

10
10
CPU times: user 14min 40s, sys: 15min 33s, total: 30min 14s
Wall time: 42.5 s


In [46]:
%%time
df_per_template = []
matches_per_template = []

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    submitted_jobs = [executor.submit(_parallel_func, template) for template in all_templates]
    results = [job.result() for job in submitted_jobs]
    
    for r in results:
        df_per_template.append(r[0])
        matches_per_template.append(r[1])
print(len(matches_per_template))
print(len(matches_per_template[0]))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 10min 31s, sys: 8min 39s, total: 19min 10s
Wall time: 44.4 s


In [47]:
%%time
df_per_template = []
matches_per_template = []
for template in tqdm(all_templates):
    base_prediction_options['template_pos'] = template
    df, matches = track_using_fdnc_ONLY_TRAINING(project_data, base_prediction_options, match_confidence_threshold, num_frames=num_templates)
    df_per_template.append(df)
    matches_per_template.append(matches)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 13min 49s, sys: 16.1 s, total: 14min 5s
Wall time: 1min 7s


In [48]:
%%time
df_per_template = []
matches_per_template = []

with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
    submitted_jobs = [executor.submit(_parallel_func, template) for template in all_templates]
    results = [job.result() for job in submitted_jobs]
    
    for r in results:
        df_per_template.append(r[0])
        matches_per_template.append(r[1])

KeyboardInterrupt: 

KeyboardInterrupt: 

# Use leifer template to get putative names

In [166]:
from DLC_for_WBFM.utils.xinwei_fdnc.predict import load_prediction_options
from fDNC.src.DNC_predict import pre_matt, predict_label, predict_matches
from DLC_for_WBFM.utils.projects.utils_neuron_names import int2name
from collections import defaultdict

In [167]:
prediction_options, template, template_label = load_prediction_options()

In [95]:
# Just convert the training data to names (not all frames)

In [116]:

# all_top5_dict = defaultdict(list)
# all_only_top_dict = defaultdict(list)
# num_templates = project_data.reindexed_metadata_training.num_frames

# for i_template in tqdm(range(num_templates)):
#     pts = project_data.get_centroids_as_numpy_training(i_template)
#     labels = predict_label(test_pos=pts, template_pos=template, template_label=template_label, **prediction_options)
#     template_top5 = labels[1]
    
#     num_candidates = 5

#     for i, neuron_top5 in enumerate(template_top5):
#         name = int2name(i+1)
#         sub_names = list(range(num_candidates))
#         neuron_key = (name, )
#         for i_candidate in range(num_candidates):
#             candidate_key = neuron_key + (i_candidate, )
#             for i_name_or_conf in range(2):
#                 if i_name_or_conf==0:
#                     key = candidate_key + ('name', )
#                 else:
#                     key = candidate_key + ('likelihood', )
#                 val = neuron_top5[i_candidate][i_name_or_conf]
#                 all_top5_dict[key].append(val)
#                 if i_candidate == 0:
#                     key = key[0,2]
#                     all_only_top_dict[key].append(val)

# # df = pd.DataFrame(all_dict, index=[0])
# df5 = pd.DataFrame(all_top5_dict)
# df1 = pd.DataFrame(all_only_top_dict)

  0%|          | 0/10 [00:00<?, ?it/s]

In [168]:
all_only_top_dict = defaultdict(list)
num_templates = project_data.reindexed_metadata_training.num_frames

for i_template in tqdm(range(num_templates)):
    pts = project_data.get_centroids_as_numpy_training(i_template)
    pts = zimmer2leifer(pts)
    labels = predict_label(test_pos=pts, template_pos=template, template_label=template_label, **prediction_options)
    template_top1 = labels[0]

    for i_neuron, neuron_top_candidate in enumerate(template_top1):
        name = int2name(i_neuron+1)
        neuron_key = (name, )
        for i_name_or_conf in range(2):
            if i_name_or_conf==0:
                key = neuron_key + ('name', )
            else:
                key = neuron_key + ('likelihood', )
            val = neuron_top_candidate[i_name_or_conf]
            all_only_top_dict[key].append(val)

df1 = pd.DataFrame(all_only_top_dict)

  0%|          | 0/10 [00:00<?, ?it/s]

In [169]:
df1

neuron_001            neuron_002            neuron_003             \
        name likelihood       name likelihood       name likelihood   
0              0.023176       AWAR   0.085650              0.987076   
1              0.032906              0.080329       AVAR   0.997236   
2              0.329082       AWAR   0.572790              0.976606   
3              0.945163       AWAR   0.995136       AVAR   0.999990   
4              0.998380       AWAR   0.849006       AVAR   0.997956   
5              0.999665       AWAR   0.944810       AVAR   0.999210   
6              0.998931       ASER   0.700420       AVAR   0.978411   
7              0.987724       ASER   0.593533       AVAR   0.946832   
8              0.965484       ASER   0.633699       AFDR   0.031765   
9              0.010058       ASER   0.775891              0.111918   

  neuron_004            neuron_005             ... neuron_116             \
        name likelihood       name likelihood  ...       name likelihood   
0       ASHL   0.090837       AVDR   0.999914  ...       AVAL   0.999222   
1       ASHL   0.178929       AVDR   0.999969  ...       AVAL   0.999873   
2       ASHL   0.947157       AVDR   0.999463  ...       AVEL   0.904060   
3       ASHL   0.747061       AVDR   0.999687  ...       AVAL   0.999824   
4       ASEL   0.025400       AVDR   0.999828  ...       AVEL   1.000000   
5       AWAL   0.964365       AVDR   0.994892  ...       AVAL   0.999172   
6       AWAL   0.980261              0.413217  ...       AVAL   0.998701   
7       AWAL   0.990574              0.702032  ...       AVAL   0.983646   
8       AWAL   0.938154              0.717761  ...       AVAL   0.960533   
9       AWAL   0.602933              0.628345  ...       AVAL   0.912368   

  neuron_117            neuron_118            neuron_119             \
        name likelihood       name likelihood       name likelihood   
0              0.026363      IL2DR   0.999759              0.011920   
1              0.185598      IL1DR   0.006079              0.052935   
2              0.997262      IL2DR   0.313265              0.002763   
3              0.045976      IL1DR   0.999993              0.574828   
4              0.583538      IL1DR   0.998634              0.277075   
5              0.901490      IL1DR   0.005085              0.003928   
6              0.855018       OLLR   0.999769              0.047571   
7              0.035192       OLLR   0.997548        ALA   0.017137   
8              0.037762              0.004685              0.130703   
9              0.082425       OLLR   0.880843              0.090704   

  neuron_120             
        name likelihood  
0      SMDDL   0.838886  
1      CEPVL   0.086580  
2      SMDDL   0.700700  
3      SMDDL   0.983317  
4      SMDDL   0.998468  
5      SMDDL   0.950200  
6      CEPVL   0.954131  
7      URAVL   0.350184  
8      URAVL   0.711316  
9      CEPVL   0.813478  

[10 rows x 240 columns]

In [170]:
# import dtale

# dtale.show(df1)

In [187]:
raw_names = list(df1.columns.levels[0])

all_match_dict = {}
all_conf_dict = {}

for n in raw_names:

    this_df = df1[n]
    candidate_names = this_df['name'].unique()
    candidate_conf = np.zeros_like(candidate_names)

    conf_func = lambda x: sum(x) / df1.shape[0]

    for i, c in enumerate(candidate_names):
        ind = this_df['name'] == c
        candidate_conf[i] = conf_func(this_df['likelihood'].loc[ind])

    # Just take the max value
    i_max = np.argmax(candidate_conf)
    this_name = candidate_names[i_max]
    if this_name != '':
        all_match_dict[n] = this_name
    else:
        all_match_dict[n] = n
    all_conf_dict[n] = candidate_conf[i_max]

In [212]:
# df_out = pd.DataFrame(index=[all_match_dict.keys()], data=[all_conf_dict.values(), all_match_dict.values()])
df_out1 = pd.DataFrame.from_dict(all_match_dict, orient='index', columns=['name'])
df_out2 = pd.DataFrame.from_dict(all_conf_dict, orient='index', columns=['likelihood'])
df_out = pd.concat([df_out1, df_out2], axis=1)

In [214]:
df_out.to_csv('test.csv')

# Get the names for the neurons in the training data

In [144]:
from DLC_for_WBFM.utils.xinwei_fdnc.predict import load_prediction_options
from fDNC.src.DNC_predict import pre_matt, predict_label, predict_matches
from DLC_for_WBFM.utils.projects.utils_neuron_names import int2name
from collections import defaultdict
from DLC_for_WBFM.utils.xinwei_fdnc.predict import combine_multiple_template_matches

In [140]:
prediction_options, template, template_label = load_prediction_options()
match_confidence_threshold = 0.1

In [141]:
matches = track_using_fdnc(project_data,
                         prediction_options,
                         template,
                         match_confidence_threshold,
                          full_video_not_training=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [135]:
template_label

array(['I1R', 'I1L', 'AMSOL', 'AMSOR', 'IL2VL', 'I2R', 'I2L', '', 'I3',
       'M3R', 'M3L', '', 'IL2DL', 'IL1VL', 'OLLL', 'IL2VR', 'URADL',
       'IL2DR', 'IL1VR', 'OLLR', 'IL1DL', 'URBL', 'URYVL', 'IL1DR',
       'URYDL', 'URYVR', 'URAVR', 'URBR', 'OLQVL', 'URAVL', 'OLQVR',
       'OLQDL', 'URYDR', 'OLQDR', 'RMEL', 'CEPVL', 'RMED', '', 'RMER',
       'SMDDL', 'CEPVR', '', 'RMDVR', 'SAAVR', 'RMDVL', 'SAAVL', 'URXL',
       'AVAR', 'CEPDL', 'SMDVR', 'AVAL', '', '', 'URXR', '', 'AVER',
       'AFDL', 'CEPDR', '', 'AFDR', '', 'AVEL', '', 'ASGR', '', '', '',
       '', 'ALA', 'AWAR', 'ASHL', 'ASHR', 'AWAL', 'ASER', '', 'ASGL', '',
       'AVHR', '', 'AVDR', 'AVJR', '', 'ASEL', '', '', '', 'AVHL', 'RIVR',
       '', 'AVJL', 'AVDL', 'RIVL', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', ''], dtype='<U5')

In [159]:
formatted_matches = [[m] for m in matches]

In [161]:
final_matches = combine_multiple_template_matches(formatted_matches)

In [162]:
final_matches

[[(0, 82, 0.4495253116882295),
  (1, 76, 0.37007150380593834),
  (3, 57, 0.2523733413266211),
  (4, 55, 0.38780313439560027),
  (5, 37, 0.19443571890377612),
  (6, 64, 0.36496252609822444),
  (7, 59, 0.3106561278671764),
  (8, 92, 0.2939097567642998),
  (9, 47, 0.29130571231512337),
  (12, 89, 0.5367727536677025),
  (13, 18, 0.532127051109205),
  (14, 26, 0.4596465290129981),
  (16, 66, 0.32005341766540707),
  (17, 36, 0.3792585263677911),
  (19, 35, 0.5360513095009017),
  (20, 19, 0.29122395566397086),
  (21, 24, 0.6006764287304671),
  (22, 102, 0.23255701156552014),
  (23, 117, 0.19724341767889883),
  (24, 14, 0.4411829744900122),
  (25, 50, 0.2762231219554822),
  (26, 107, 0.2644773254234498),
  (27, 44, 0.33890519841191225),
  (28, 20, 0.3524002552039589),
  (30, 51, 0.24584277091556764),
  (31, 63, 0.2749804455606422),
  (32, 114, 0.4122230061534514),
  (33, 49, 0.5918621679289955),
  (34, 27, 0.7615081837235786),
  (36, 60, 0.7484437806420352),
  (37, 41, 0.3345911090961652),
  (

# Scratch: try some purely classical matching

In [19]:
import open3d as o3d
from probreg import bcpd, cpd
from probreg import callbacks
from DLC_for_WBFM.utils.point_clouds.utils_bcpd import match_2vol_BCPD, match_2vol_rigid
from DLC_for_WBFM.utils.xinwei_fdnc.formatting import zimmer2leifer

In [160]:

source = o3d.geometry.PointCloud()
target = o3d.geometry.PointCloud()

pts0 = project_data.get_centroids_as_numpy_training(0)
source.points = o3d.utility.Vector3dVector(pts0)
pts1 = project_data.get_centroids_as_numpy_training(1)
target.points = o3d.utility.Vector3dVector(pts1)


In [2]:
# to_plot = True
# if to_plot:
#     cbs = [callbacks.Open3dVisualizerCallback(source, target)]
# tf_param = bcpd.registration_bcpd(source, target, w=1e-12,
#                                   maxiter=100,
#                                   callbacks=cbs)

In [30]:

bcpd_kwargs = {'lmd': 0.1}

pts0 = zimmer2leifer(project_data.get_centroids_as_numpy_training(5))
pts1 = zimmer2leifer(project_data.get_centroids_as_numpy_training(4))
match_2vol_BCPD(pts0, pts1, w=1e-10, bcpd_kwargs=bcpd_kwargs)
# match_2vol_rigid(pts0, pts1)

(array([[  0, 115],
        [  1,   1],
        [  2,   1],
        [  3,  97],
        [  4, 112],
        [  5, 112],
        [  6, 112],
        [  7, 115],
        [  8, 115],
        [  9, 112],
        [ 10,  97],
        [ 11,  69],
        [ 12,  97],
        [ 13, 115],
        [ 14,  23],
        [ 15, 115],
        [ 16,  49],
        [ 17, 115],
        [ 18, 117],
        [ 19,  23],
        [ 20,  23],
        [ 21,  97],
        [ 22, 115],
        [ 23,  49],
        [ 24,  24],
        [ 25,  90],
        [ 26, 117],
        [ 27,  49],
        [ 28,  90],
        [ 29, 112],
        [ 30,  52],
        [ 31, 112],
        [ 32, 112],
        [ 33,  62],
        [ 34,  52],
        [ 35, 117],
        [ 36, 117],
        [ 37, 117],
        [ 38,  97],
        [ 39,  62],
        [ 40,  49],
        [ 41,  23],
        [ 42, 115],
        [ 43, 115],
        [ 44,  23],
        [ 45, 115],
        [ 46,  29],
        [ 47,  92],
        [ 48,  90],
        [ 49,  49],
